<a href="https://colab.research.google.com/github/xuanmse/python-for-engineer/blob/master/Exercise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| Thông tin   |               |
| ----------- | -----------   |
| MSSV        | 22mse23076    |
| Họ Và Tên   | KIEU VAN XUAN |

Bài tập 2: Tự lập trình vẽ lại các biểu đồ theo gợi ý ở đây [M5 EDA | Kaggle](https://www.kaggle.com/code/akshaychaudhary44/m5-eda)

#**Cài đặt môi trường & tải tập dữ liệu**

##**Cài đặt thư viện**

In [1]:
!pip install kaggle # cài đặt công cụ Kaggle
!pip install seaborn
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##**Cài đặt xác thực**

In [2]:
# Tải thông tin tài khoảng để xác thực trên kaggle và lưu vào máy ảo colab
! yes | mkdir /root/.kaggle
! yes | wget https://raw.githubusercontent.com/xuanmse/python-for-engineer/master/kaggle.json -P /root/.kaggle/
! chmod 600 /root/.kaggle/kaggle.json

--2022-11-01 16:11:18--  https://raw.githubusercontent.com/xuanmse/python-for-engineer/master/kaggle.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68 [text/plain]
Saving to: ‘/root/.kaggle/kaggle.json’

kaggle.json         100%[===================>]      68  --.-KB/s    in 0s      

2022-11-01 16:11:18 (2.36 MB/s) - ‘/root/.kaggle/kaggle.json’ saved [68/68]



##**Giải nén**

In [ ]:
!yes | kaggle competitions download -c m5-forecasting-accuracy -p /kaggle/input/
!yes | unzip /kaggle/input/m5-forecasting-accuracy.zip -d /kaggle/input/m5-forecasting-accuracy
!yes | rm /kaggle/input/m5-forecasting-accuracy.zip

##**Nhập thư viện và kiểm tra dữ liệu**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

##**Tải tập dữ liệu lên ô nhớ**

In [ ]:
sale2 = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
cale4 = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
prices = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sell_prices.csv")

## **Thay đỗi thông tin tên cột của tập dữ liệu**

In [ ]:
import datetime
x = datetime.date(2011,1,29)
for c in ((sale2.columns)[6:]):
    sale2.rename(columns={c:x},inplace=True)
    x = x + datetime.timedelta(days=1)



# **Trực quan hóa dữ liệu**

##**Doanh số năm mặt hàng đầu tiên theo thời gian**

In [ ]:
X = sale2.groupby('item_id')[(sale2.columns)[6:]].sum()
fig = px.line(X[:5].transpose(), title='Doanh số năm mặt hàng đầu tiên theo thời gian')
fig.show()

##**Tổng doanh số bán hàng theo thời gian**

In [ ]:
fig = px.line(X.sum().transpose(), title='Doanh số năm mặt hàng đầu tiên theo thời gian')
fig.show()

##Doanh số hàng tháng ở các thành phố

In [ ]:


X = sale2
X = X.groupby(X.columns.map(lambda x : x if type(x) == str else x.strftime('%Y-%m')),axis=1).sum()
for c in ((X.columns)[:-6]):
    x= datetime.datetime.strptime(c, '%Y-%m').date()
    X.rename(columns={c:x},inplace=True)



In [ ]:
F = X.groupby(['state_id'])[(X.columns)[:-6]].sum()
fig = px.line(F.transpose()[1:], title='Doanh số hàng tháng ở các thành phố')
fig.show()
# plt.figure(num=None, figsize=(7, 4), dpi=80, facecolor='w', edgecolor='k')
# for i in range(3):
#     plt.plot(F.iloc[i,1:-1],label=F.index[i])
#     plt.legend(loc='best')
# plt.show()

##**Doanh số theo tháng của các doanh mục**

In [ ]:
F = X.groupby(['cat_id'])[(X.columns)[:-6]].sum()
fig = px.line(F.transpose()[1:], title='Doanh số theo tháng của các doanh mục')
fig.show()

##**Tổng doanh số bán hàng của các danh mục**

In [ ]:
F = X.groupby(['cat_id'])[(X.columns)[:-6]].sum().transpose().sum()
fig = px.bar(F, title='Tổng doanh số bán hàng của các danh mục')
fig.show()

##**Doanh số hàng tháng theo cửa hàng**

In [ ]:
def visualize_store_sales_monthly(DF, fig,row,col ):
  for i in range(len(DF.index)):
    tmp = DF.iloc[i]
    fig.add_trace(go.Scatter(x=tmp.index[1:], y = tmp.values[1:],name=tmp.name),row = row, col =col)

F = X.groupby(['store_id'])[(X.columns)[:-6]].sum()

fig = make_subplots(rows=1, cols=3, shared_yaxes=True)
visualize_store_sales_monthly (F[F.index.str.startswith("CA")] ,fig, 1,1)
visualize_store_sales_monthly (F[F.index.str.startswith("TX")] ,fig, 1,2)
visualize_store_sales_monthly (F[F.index.str.startswith("WI")] ,fig, 1,3)

fig.show()

##**Doanh số hàng tháng theo bộ phận khác nhau của ba bang**

In [ ]:
import matplotlib.pyplot as plt
F = X.groupby(['state_id','dept_id'])[(X.columns)[6:]].sum()

fig = make_subplots(rows=3, cols=7, shared_yaxes=True)

for i in range(3):
    for j in range(7):
        k = (7*i)+(j)
        tmp = F.iloc[k,1:-1]
        fig.add_trace(go.Scatter(x=tmp.index[1:], y = tmp.values[1:], name = tmp.name[0] +":" + tmp.name[1] ),row = i+1, col =j+1)
fig.show()    


##**Tổng doanh số với đường làm mịn**

In [ ]:
F = sale2.drop(labels=[datetime.date(2011, 12, 25),datetime.date(2012, 12, 25), datetime.date(2013, 12, 25),datetime.date(2014, 12, 25),datetime.date(2015, 12, 25)],axis=1)

In [ ]:
S = F.iloc[:,6:].sum()

import statsmodels.api as sm
lowess = sm.nonparametric.lowess
z = lowess(S.values, range(S.shape[0]))
L = pd.Series(z[:,1],S.index)

fig = go.Figure()
fig.add_trace(go.Scatter(x=S.index[1:], y = S.values[1:],
                    mode='lines',
                    name='total'))
fig.add_trace(go.Scatter(x=L.index[1:], y = L.values[1:],
                    mode='lines',
                    name='smoothing'))
fig.show()    

# results = px.get_trendline_results(fig)
# print(results)

# results.query("sex == 'Male' and smoker == 'Yes'").px_fit_results.iloc[0].summary()

##**Doanh số dư theo mùa**

In [ ]:
M = F.iloc[:,6:].sum()
z = lowess(M, range(M.shape[0]))
Z = M.values - z[:,1]
A = (pd.DataFrame(data=Z,index=M.index)).T
A = A.groupby(A.columns.map(lambda x : str(x.weekday())+'-'+str(x.month)),axis=1).sum()
H = np.zeros((7,12))
for i in range(7):
    for j in range(12):
        H[i,j] = A[str(i)+'-'+str(j+1)]//1000
fig = px.imshow(H, x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                y=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday','Sunday'])
fig.show()

##**Doanh số theo mùa tại các bang**

In [ ]:
def visualize_sale_state_season(df_tmp, se_tmp ,row,col):
  fig.add_trace(go.Scatter(x=df_tmp.index[1:], y = df_tmp.values[1:],name="total"),row = row, col =col)
  fig.add_trace(go.Scatter(x=se_tmp.index[1:], y = se_tmp.values[1:],name="smooth"),row = row, col =col)

M = F.groupby(['state_id'])[(F.columns)[6:]].sum()
fig = make_subplots(rows=1, cols=3, shared_yaxes=True, column_titles =["CA","TX", "WI"])
for i in range (3):
  z = lowess(M.iloc[i,:].values, range(M.shape[1]))
  visualize_sale_state_season (M.iloc[i,:],pd.Series(z[:,1],M.columns), 1,i+1)
fig.show()


##**Hàng Tuần**

In [ ]:
mean_sales = F.iloc[:,6:].sum().mean()

In [ ]:
A = M
for i in range(3):
    z = lowess(M.iloc[i,:].values, range(M.shape[1]))
    A.iloc[i,:] = M.iloc[i,:] - z[:,1]
A = A/mean_sales
A = A.groupby(A.columns.map(lambda x : str(x.weekday())),axis=1).sum()
px.line(A.transpose(), title='Doanh số hàng tuần theo mùa',)

##**Hàng tháng**


In [ ]:
A = M
for i in range(3):
    z = lowess(M.iloc[i,:].values, range(M.shape[1]))
    A.iloc[i,:] = M.iloc[i,:] - z[:,1]
A = A/mean_sales
A = A.groupby(A.columns.map(lambda x : (x.month)),axis=1).sum()
px.line(A.transpose(), title='Doanh số hàng tuần theo mùa',)
# plt.figure(num=None, figsize=(7,3), dpi=80, facecolor='w', edgecolor='k')
# for i in range(3):
#     plt.plot(A.iloc[i,:],label=A.index[i])
#     plt.legend(loc='best')
#     plt.xticks(range(1,13),['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
# plt.show()



##**Hàng tuần, Hàng tháng theo danh mục**

In [ ]:
M = F.groupby(['cat_id','state_id'])[(F.columns)[6:]].sum()
A = M
for i in range(9):
    z = lowess(M.iloc[i,:].values, range(M.shape[1]))
    A.iloc[i,:] = M.iloc[i,:] - z[:,1]
G = A/mean_sales
A = G.groupby(G.columns.map(lambda x : str(x.weekday())),axis=1).sum()
B = G.groupby(G.columns.map(lambda x : x.month),axis=1).sum()
def visualize_sale_state_season1(df_tmp,name,color,row,col):
  fig.add_trace(go.Scatter(x=df_tmp.index[1:], y = df_tmp.values[1:],name=name,marker=dict(color=color)),row = row, col =col)

fig = make_subplots(rows=2, cols=3, column_titles =["FOODS","HOBBIES", "HOUSEHOLD"])

for i in range (9):
  visualize_sale_state_season1 (A.iloc[i,:],"%s-%s"%(A.iloc[i,:].name),i%3, 1,i//3+1)
  visualize_sale_state_season1 (B.iloc[i,:],"%s-%s"%(B.iloc[i,:].name),i%3, 2,i//3+1)
fig.show()


##**Ngày có sự kiện và không có sự kiện**


In [ ]:
Non_events = cale4.date.count() - cale4.event_name_1.count()
Events = cale4.event_name_1.count()
fig = px.pie(values=[Non_events,Events],names=["Non_events","Events"])
fig.show()


Số loại sự kiện


In [ ]:
E = cale4.groupby(['event_type_1']).count().iloc[:,1].sort_values(ascending=True)

fig = px.bar(E, x=E.index, y=E.values)
fig.show()


##**Phân bổ giá sản phẩm của các bộ phận khác nhau theo bang(tobe update)**

In [ ]:
A=prices.groupby([prices.store_id.str.slice(0,-2),prices.item_id.str.slice(0,-4)])
B = []
for name, group in A:
    B.append(name)

In [ ]:
import plotly.figure_factory as ff
def get_r_c(b):
  c = ["CA","TX", "WI"].index(b[0])
  r = 0
  if b[1].startswith("FOODS"):
    r = 0
    ylabel = "FOODS"
  elif b[1].startswith("HOBBIES"):
    r = 1
    ylabel = "HOBBIES"
  elif b[1].startswith("HOUSEHOLD"):
    r = 2
    ylabel = "HOUSEHOLD"
  return r,c, b[0], ylabel

fig, ax = plt.subplots(3,3,sharey=True,sharex=True, figsize=(15,7))
for b in B:
  i,j,xlabel,ylabel = get_r_c(b)
  sns.kdeplot(np.log10(A.get_group(b)['sell_price']),bw_adjust=3,legend=False,shade=True,ax=ax[i,j])
  ax[i,j].grid(b=True,axis='y')
  ax[i,j].set(xlabel = xlabel, ylabel =ylabel)
fig.canvas.draw()
loc,labels = plt.xticks()
for ax in ax.flat:
    ax.label_outer()
    ax.set_xlim(-1,2)
    ax.set_xticklabels([10 ** int(i.get_text().replace(u'\u2212', '-')) for i in labels])
fig.subplots_adjust( hspace = 0.0,wspace=0.1)
plt.show()


##**Phân bổ giá của sản phầm theo phòng ban theo năm** (TOBE Update)

In [ ]:
A=prices.groupby([prices.item_id.str.slice(0,-4),prices.wm_yr_wk//100 + 1900])['sell_price']
n = []
for name,group in A:
    n.append(name)

In [ ]:
fig, ax = plt.subplots(6,3,sharey=True,sharex=True, figsize=(10,7))

for i in n:
    j = 0 if i[0][:-2]=='FOODS' else (1 if i[0][:-2]=='HOBBIES' else 2)
    k = i[1] - 2011
    sns.kdeplot(np.log10(A.get_group(i)),legend=False,shade=True,bw_adjust=3,ax=ax[k,j])
    ax[k,j].set(xlabel = i[0][:-2], ylabel =k+2011)

fig.canvas.draw()
loc,labels = plt.xticks()
for ax in ax.flat:
    ax.label_outer()
    ax.set_xlim(-1,2)
    ax.set_xticklabels([10 ** int(i.get_text().replace(u'\u2212', '-')) for i in labels])
plt.yticks([], [])
fig.subplots_adjust( hspace = 0.0,wspace=0.2)
plt.show()

##**Doanh số theo doanh mục trong những ngày có sự kiện và không có sự kiện**

In [ ]:


c = [datetime.date(2011, 12, 25),datetime.date(2012, 12, 25), datetime.date(2013, 12, 25),datetime.date(2014, 12, 25),datetime.date(2015, 12, 25)]
F = sale2.drop(labels=c,axis=1)
event_dates = cale4[pd.notnull(cale4['event_name_1'])]['date'] 
event_dates = [datetime.datetime.strptime(i, '%Y-%m-%d').date() for i in event_dates.values]

for i in c:
    event_dates.remove(i)
event_sales = F[['id','dept_id','cat_id', 'store_id', 'state_id']+event_dates[:-8]]
nevent_sales = F[F.drop(columns=event_dates[:-8]).columns]



In [ ]:
N3 = []
L1 = []
L2 = []
L3 = []
means = []
e = event_sales.groupby(['cat_id']).sum()
ne = nevent_sales.groupby(['cat_id']).sum()
t = F.groupby(['cat_id']).sum()
means = t.mean(axis=1)
for i in range(3):
    n1 = lowess(e.iloc[i,:].values, range(e.shape[1]),frac=0.4)
    n2 = lowess(ne.iloc[i,:].values, range(ne.shape[1]),frac=0.4)
    n3 = lowess(t.iloc[i,:].values, range(t.shape[1]),frac=0.4)
    L1.append(n1[:,1])
    L2.append(n2[:,1])
    L3.append(n3[:,1])
    n3 = (t.iloc[i,:] - n3[:,1]) /(means[i])
    N3.append(n3)
N3 = pd.DataFrame(N3,columns = t.columns)
L1 = pd.DataFrame(L1,columns = e.columns)
L2 = pd.DataFrame(L2,columns = ne.columns)
L3 = pd.DataFrame(L3,columns = t.columns)

In [ ]:

fig = make_subplots(rows=1, cols=3,column_titles =["FOODS","HOBBIES", "HOUSEHOLD"])
for j in range (3):

  ne_tmp = ne.iloc[j,:] 
  fig.add_trace(go.Scatter(x=ne_tmp.index, y = ne_tmp.values,name="non-event", marker=dict(color='gainsboro')),row = 1, col =j+1)
  e_tmp = e.iloc[j,:]
  fig.add_trace(go.Scatter(x=e_tmp.index, y = e_tmp.values,name="event", marker=dict(color='lightcoral')),row = 1, col =j+1)
  L3_tmp = L3.iloc[j,:]
  fig.add_trace(go.Scatter(x=L3_tmp.index, y = L3_tmp.values,name="l-gen", marker=dict(color='black'), line= dict(dash = "dash",width=3)),row = 1, col =j+1)
  L1_tmp = L1.iloc[j,:]
  fig.add_trace(go.Scatter(x=L1_tmp.index, y = L1_tmp.values,name="l-event", marker=dict(color='red'), line= dict(width=3)),row = 1, col =j+1)
  L2_tmp = L2.iloc[j,:]
  fig.add_trace(go.Scatter(x=L2_tmp.index, y = L2_tmp.values,name="l-non-event", marker=dict(color='darkgray'), line= dict(width=3)),row = 1, col =j+1)
  
fig.show()

In [ ]:
N1 = N3[event_dates[:-8]].T.assign(Event='True')
N2 = N3[N3.drop(columns=event_dates[:-8]).columns].T.assign(Event='False')
cdf = pd.concat([N1,N2])
mdf = pd.melt(cdf, id_vars=['Event'], var_name=['Letter'])

In [ ]:
fig = px.box(mdf, x="value", y="Letter",color="Event")
fig.show()

In [ ]:
N3 = []
means = []
t = F.groupby(['cat_id']).sum()
means = t.mean(axis=1)
for i in range(3):
    n3 = lowess(t.iloc[i,:].values, range(t.shape[1]),frac=0.5)
    n3 = (t.iloc[i,:] - n3[:,1]) /(means[i])
    N3.append(n3)
N3 = pd.DataFrame(N3,columns = t.columns)

c = [datetime.date(2011, 12, 25),datetime.date(2012, 12, 25), datetime.date(2013, 12, 25),datetime.date(2014, 12, 25),datetime.date(2015, 12, 25)]
F = sale2.drop(labels=c,axis=1)
event_dates = cale4[pd.notnull(cale4['event_name_1'])][['date','event_type_1']] 
event_dates['date'] = [datetime.datetime.strptime(i, '%Y-%m-%d').date() for i in event_dates['date']]

for i in c:
    event_dates = event_dates[event_dates['date'] != i]


s = N3.T
s.reset_index(drop=False, inplace=True)
s.rename(columns = {'index':'date'},inplace=True)
s = pd.merge(left=event_dates, right=s, how='left', left_on='date', right_on='date')

s = s.groupby(['event_type_1']).median()
s.reset_index(drop=False, inplace=True)
s = s.melt(id_vars=['event_type_1'], value_vars=s.columns[1:])

In [ ]:
fig = px.histogram(x=s.value,y=s.variable, color=s.event_type_1,barmode='group')
fig.show()

##**Doanh số theo bang vào ngày SNAP & không SNAP**

In [ ]:


c = [datetime.date(2011, 12, 25),datetime.date(2012, 12, 25), datetime.date(2013, 12, 25),datetime.date(2014, 12, 25),datetime.date(2015, 12, 25)]
F = sale2.drop(labels=c,axis=1)
F = F.groupby(['state_id']).sum()
CA_snap = cale4.date[cale4.snap_CA == 1].values
TX_snap = cale4.date[cale4.snap_TX == 1].values
WI_snap = cale4.date[cale4.snap_WI == 1].values
CA_snap = [datetime.datetime.strptime(i, '%Y-%m-%d').date() for i in CA_snap]
TX_snap = [datetime.datetime.strptime(i, '%Y-%m-%d').date() for i in TX_snap]
WI_snap = [datetime.datetime.strptime(i, '%Y-%m-%d').date() for i in WI_snap]

CA_snap_sales = F[CA_snap[:-20]].iloc[0,:]
TX_snap_sales = F[TX_snap[:-20]].iloc[1,:]
WI_snap_sales = F[WI_snap[:-20]].iloc[2,:]

CA_nsnap_sales = F[F.drop(columns=CA_snap[:-20]).columns].iloc[0,:]
TX_nsnap_sales = F[F.drop(columns=TX_snap[:-20]).columns].iloc[1,:]
WI_nsnap_sales = F[F.drop(columns=WI_snap[:-20]).columns].iloc[2,:]



In [ ]:
N3 = []
L1 = []
L2 = []
L3 = []
means = []
e = [CA_snap_sales,TX_snap_sales,WI_snap_sales]
ne = [CA_nsnap_sales,TX_nsnap_sales,WI_nsnap_sales]
t = F
means = t.mean(axis=1)
for i in range(3):
    n1 = lowess(e[i].values, range(len(e[i])),frac=0.5)
    n2 = lowess(ne[i].values, range(len(ne[i])),frac=0.5)
    nn3 = lowess(t.iloc[i,:].values, range(t.shape[1]),frac=0.5)
    n1 = pd.Series(n1[:,1],e[i].index)
    n2 = pd.Series(n2[:,1],ne[i].index)
    n3 = pd.Series(nn3[:,1],t.columns)
    L1.append(n1)
    L2.append(n2)
    L3.append(n3)
    n3 = (t.iloc[i,:] - nn3[:,1]) /(means[i])
    N3.append(n3)
N3 = pd.DataFrame(N3,columns = t.columns)

In [ ]:
fig = make_subplots(rows=1, cols=3,column_titles =["CA","TX", "WI"])
for j in range (3):

  ne_tmp = ne[j] 
  fig.add_trace(go.Scatter(x=ne_tmp.index, y = ne_tmp.values,name="non-event", marker=dict(color='gainsboro')),row = 1, col =j+1)
  e_tmp = e[j]
  fig.add_trace(go.Scatter(x=e_tmp.index, y = e_tmp.values,name="event", marker=dict(color='lightcoral')),row = 1, col =j+1)
  L3_tmp = L3[j]
  fig.add_trace(go.Scatter(x=L3_tmp.index, y = L3_tmp.values,name="l-gen", marker=dict(color='black'), line= dict(dash = "dash",width=3)),row = 1, col =j+1)
  L1_tmp = L1[j]
  fig.add_trace(go.Scatter(x=L1_tmp.index, y = L1_tmp.values,name="l-event", marker=dict(color='red'), line= dict(width=3)),row = 1, col =j+1)
  L2_tmp = L2[j]
  fig.add_trace(go.Scatter(x=L2_tmp.index, y = L2_tmp.values,name="l-non-event", marker=dict(color='darkgray'), line= dict(width=3)),row = 1, col =j+1)
  
fig.show()

##**Phần trăm doanh số theo ngày SNAP và không SNAP**

In [ ]:
perc = []
for i in range(3):
    a = e[i].sum()/(1908/3)
    b = ne[i].sum()/(2*1908/3)
    perc.append([a*100/(a+b), b*100/(a+b)])

perc = pd.DataFrame(perc,index=["CA","TX","WI"],columns=["SNAP","NON_SNAP"])
perc.reset_index(drop=False, inplace=True)
s = perc.melt(id_vars=['index'], value_vars=perc.columns[1:])
s = s.sort_values(by='index',ignore_index=True)

In [ ]:
fig = px.histogram(x=s['index'],y=s.value, color=s.variable,barmode='group', text_auto='.2f')
fig.show()

##**Phần trăm doanh số theo ngày SNAP & không SNAP theo danh mục**

In [ ]:
F  = sale2.groupby(['state_id','cat_id']).sum()
F.reset_index(drop=False, inplace=True)
l = [CA_snap[:-20],TX_snap[:-20],WI_snap[:-20]]

t = []
for i in range(9):
    t.append((F['state_id'][i],F['cat_id'][i]))
    
a=[]
b = []
perc = []
for i in range(9):
    a.append(F[l[i//3]].loc[i,:].sum()/(1908/3))
    b.append(F[F.drop(columns=l[i//3]).columns].iloc[i,3:].sum()/(2*1908/3))
    perc.append([a[i]*100/(a[i]+b[i]), b[i]*100/(a[i]+b[i])])
perc = pd.DataFrame(perc,index=t,columns=["SNAP","NON_SNAP"])
perc.reset_index(drop=False, inplace=True)
s = perc.melt(id_vars=['index'], value_vars=perc.columns[1:])
s = s.sort_values(by='index',ignore_index=True)

In [ ]:
s['index']
fig = px.histogram(x=s['index'].map(str),y=s.value, color=s.variable,barmode='group', text_auto='.2f')
fig.show()

# fig.show()

##**Tác động của ngày SNAP đến doanh số doanh mục thức ăn của CA**

In [ ]:
z = F.drop(labels=c,axis=1).iloc[0,2:]
n1 = lowess(z.values, range(1908),frac=0.5)
rel_sales = (z.values - n1[:,1]) / z.mean()
rel_sales = pd.Series(rel_sales,z.index)

rel_sales1 = rel_sales[CA_snap[:-20]]
rel_sales2 = rel_sales.drop(labels = CA_snap[:-20])

a = rel_sales1.groupby(rel_sales1.index.map(lambda x : str(x.weekday())+'-'+str(x.month))).sum()
b = rel_sales2.groupby(rel_sales2.index.map(lambda x : str(x.weekday())+'-'+str(x.month))).sum()

H = np.zeros((7,12))
for i in range(7):
    for j in range(12):
        H[i,j] = a[str(i)+'-'+str(j+1)] - b[str(i)+'-'+str(j+1)]

In [ ]:
fig = px.imshow(H, x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                y=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday','Sunday'])
fig.show()

##**Doanh số Ba mặt hàng ngẫu nhiên giữa năm 2015 với điểm Màu đen là Sự kiện và Nền màu xám là ngày SNAP**

In [ ]:
x = sale2[sale2['id'].isin(["FOODS_2_092_CA_1_validation", "HOUSEHOLD_2_071_TX_2_validation", "HOBBIES_1_348_WI_3_validation"])].loc[:,datetime.date(2015,5,1):datetime.date(2015,10,1)]

In [ ]:
v = event_dates[event_dates['date'].isin([datetime.date(2015,5,1) + datetime.timedelta(days=x) for x in range(153)])]['date']
d = []
for i in range(len(v)):
    d.append(x.columns.get_loc(v.values[i]))   
c = list(set(CA_snap) & set([datetime.date(2015,5,1) + datetime.timedelta(days=x) for x in range(153)]))
e = []
for i in range(len(c)):
    e.append(x.columns.get_loc(c[i]))


In [ ]:
    
plt.figure(None, figsize=(15, 3), dpi=80, facecolor='w', edgecolor='k')
for i in range(len(c)):
    plt.axvline(c[i],color='gainsboro',linewidth=5.5)
plt.plot(x.iloc[0,:], '-gd', markevery = d,markerfacecolor='black',marker='.',markersize=10)
plt.show()